In [67]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import random

In [11]:
!head train_sample.csv

0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,3
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0,0,1,1,0,0,0,0,0,0,3
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0,0,1,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,3
0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,4,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0


In [20]:
features = ["f" + str(i) for i in range(0,99)]
label = ["action"]

headers = features + label

df = pd.read_csv('train_sample.csv', 
                 sep = ',', 
                 header = None,
                 names = headers)
df["reward"] = np.random.randint(0,3,size=(100))

In [21]:
print("headers length: {}, df lenght: {}".format(len(headers), len(df)))

headers length: 100, df lenght: 100


In [22]:
df["f0"].value_counts()

4    34
0    33
2    29
5     3
3     1
Name: f0, dtype: int64

In [23]:
df["reward"].value_counts()

0    40
2    30
1    30
Name: reward, dtype: int64

In [8]:
print("no of states without duplicates: {}".format(len(df.drop_duplicates())))

no of states without duplicates: 9137


In [24]:
y_train_all = df["reward"]
X_train_all = df.drop(columns="reward")

In [25]:
print("no of states without duplicates and action column: {}".format(len(X_train_all.drop_duplicates())))

no of states without duplicates and action column: 100


In [12]:
# X_train_all[["f0", "f1"]]

In [26]:
def evaluate_model(xtrain, ytrain, xtest, ytest, model):
    y_pred_test = model.predict(xtest)
    y_pred_train = model.predict(xtrain)
    test_accuracy = accuracy_score(ytest, y_pred_test)
    train_accuracy = accuracy_score(ytrain, y_pred_train)
    print('Training accuracy: {0:.2f}, evaluation accuracy: {1:.2f}'.format(train_accuracy, test_accuracy))


Basic logisting regresion

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, test_size=0.2)

classifier = LogisticRegression().fit(X_train, y_train)

/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [28]:
evaluate_model(X_train, y_train, X_test, y_test, classifier)

Training accuracy: 0.88, evaluation accuracy: 0.40


In [83]:
#dla kazdej akcji wywołaj predykcje
def load_historical_train():
    df = pd.read_csv('train.csv', sep = ';', encoding='utf-8')
    return df

def save_trainig_data(all_training_data):
    all_training_data.to_csv("train.csv", sep=';', encoding='utf-8', index=False)

def get_game_state():
    return all_without_action_nor_reward.iloc[[0]].copy()

def get_best_action(game_state, classifier):
    actions = [0,1,2,3,4,5]
    results = dict()
    for action in actions:
        game_state_1f = game_state
        game_state_1f['action'] = action
        result = classifier.predict(game_state_1f.values.reshape(1, -1))
        results[action] = result[0]

    max_value = -1
    max_action = 0
    for key in results.keys():
        if results[key] > max_value:
            max_value = results[key]
            max_action = key
    return max_action

def play_game(action):
    reward = random.randint(0,3) 
    return reward


def initialize_classifier(all_training_data):
    X_train = all_training_data.drop(columns="reward")
    y_train = all_training_data["reward"]
    
    mlp = MLPClassifier(solver='adam', alpha=1e-4, 
                    hidden_layer_sizes=(24, 24, 24, 24), 
                          random_state=1, verbose=True)
    
    return mlp.fit(X_train, y_train)

def retrain_model(all_training_data, game_state, action, reward):
    final_training_vector = get_game_state()
    final_training_vector['action'] = best_action
    final_training_vector['reward'] = reward

    all_training_data.append(final_training_vector)    
    return (initialize_classifier(all_training_data), all_training_data)
    


In [80]:
!head train.csv

;f0;f1;f2;f3;f4;f5;f6;f7;f8;f9;f10;f11;f12;f13;f14;f15;f16;f17;f18;f19;f20;f21;f22;f23;f24;f25;f26;f27;f28;f29;f30;f31;f32;f33;f34;f35;f36;f37;f38;f39;f40;f41;f42;f43;f44;f45;f46;f47;f48;f49;f50;f51;f52;f53;f54;f55;f56;f57;f58;f59;f60;f61;f62;f63;f64;f65;f66;f67;f68;f69;f70;f71;f72;f73;f74;f75;f76;f77;f78;f79;f80;f81;f82;f83;f84;f85;f86;f87;f88;f89;f90;f91;f92;f93;f94;f95;f96;f97;f98;action;reward
0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;0;1;1;0;1;0;0;0;1;0;0;0;0;1;0;0;1;0;1;0;0;3;1
1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;0;1;1;0;1;0;0;0;1;0;0;0;0;1;0;0;1;0;1;0;0;3;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0;0;1;0;0;1;0;0;0;0;0;0;0;0;0;0;1;2;0;0;1;1;0;0;0;0;0;0;3;2
2;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0;0;1;0;0;1;0;0;0;0;0;0;0;0;0;0;1;2;0;0;1;1;0;0;0;0;0;0;3;0;0;0;0;0;0;0;0;0;0;0;0;1;1;0;0;0;0;0;0;0;0;0;0;0;2;0;0;0;0;1;0;1;0;0;0;0;0;0;0;1;0;0;0;0

In [85]:
all_training_data = load_historical_train()
classifier = initialize_classifier(all_training_data)

# game loop starts here

game_state = get_game_state()
best_action = get_best_action(game_state, classifier)
reward = play_game(best_action)

classifier, all_training_data = retrain_model(all_training_data, game_state, best_action, reward)

save_trainig_data(all_training_data)

# save_training_data

[1]
[1]
[1]
[1]
[1]
[1]


/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [84]:
all_training_data = load_historical_train()
all_training_data

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,action,reward
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,3,1
1,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,3,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,3,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2
4,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,1,0,2
5,4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
6,2,0,0,0,0,1,1,0,1,1,...,1,0,0,1,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
8,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,4,2
9,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
pickle.dump(classifier, open("ai_model_logit.pkl","wb"))

In [17]:
classifier = LogisticRegression().fit(X_train_all, y_train_all)

xgboost

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, 
                                                    test_size=0.2, random_state=7)

xgb_model = XGBClassifier(n_estimators=500).fit(X_train, y_train,eval_set = [[X_test, y_test]], verbose = 1)

[0]	validation_0-merror:0.258611
[1]	validation_0-merror:0.258611
[2]	validation_0-merror:0.258065
[3]	validation_0-merror:0.258065
[4]	validation_0-merror:0.258065
[5]	validation_0-merror:0.258065
[6]	validation_0-merror:0.258065
[7]	validation_0-merror:0.258065
[8]	validation_0-merror:0.258065
[9]	validation_0-merror:0.258065
[10]	validation_0-merror:0.258065
[11]	validation_0-merror:0.258065
[12]	validation_0-merror:0.258611
[13]	validation_0-merror:0.258611
[14]	validation_0-merror:0.258611
[15]	validation_0-merror:0.258611
[16]	validation_0-merror:0.258611
[17]	validation_0-merror:0.258611
[18]	validation_0-merror:0.258611
[19]	validation_0-merror:0.259158
[20]	validation_0-merror:0.259158
[21]	validation_0-merror:0.259158
[22]	validation_0-merror:0.259158
[23]	validation_0-merror:0.259158
[24]	validation_0-merror:0.259158
[25]	validation_0-merror:0.259158
[26]	validation_0-merror:0.259158
[27]	validation_0-merror:0.259158
[28]	validation_0-merror:0.259158
[29]	validation_0-merror

[238]	validation_0-merror:0.266812
[239]	validation_0-merror:0.266812
[240]	validation_0-merror:0.266812
[241]	validation_0-merror:0.266266
[242]	validation_0-merror:0.266812
[243]	validation_0-merror:0.266266
[244]	validation_0-merror:0.266266
[245]	validation_0-merror:0.266266
[246]	validation_0-merror:0.266266
[247]	validation_0-merror:0.266812
[248]	validation_0-merror:0.267359
[249]	validation_0-merror:0.267359
[250]	validation_0-merror:0.266812
[251]	validation_0-merror:0.266812
[252]	validation_0-merror:0.267906
[253]	validation_0-merror:0.267906
[254]	validation_0-merror:0.267359
[255]	validation_0-merror:0.266812
[256]	validation_0-merror:0.266812
[257]	validation_0-merror:0.266812
[258]	validation_0-merror:0.267359
[259]	validation_0-merror:0.266812
[260]	validation_0-merror:0.266266
[261]	validation_0-merror:0.266266
[262]	validation_0-merror:0.266266
[263]	validation_0-merror:0.266812
[264]	validation_0-merror:0.265719
[265]	validation_0-merror:0.266266
[266]	validation_0-m

[474]	validation_0-merror:0.27392
[475]	validation_0-merror:0.274467
[476]	validation_0-merror:0.274467
[477]	validation_0-merror:0.275014
[478]	validation_0-merror:0.275014
[479]	validation_0-merror:0.274467
[480]	validation_0-merror:0.274467
[481]	validation_0-merror:0.275014
[482]	validation_0-merror:0.275014
[483]	validation_0-merror:0.275014
[484]	validation_0-merror:0.275014
[485]	validation_0-merror:0.275014
[486]	validation_0-merror:0.27392
[487]	validation_0-merror:0.27392
[488]	validation_0-merror:0.27392
[489]	validation_0-merror:0.273373
[490]	validation_0-merror:0.272827
[491]	validation_0-merror:0.272827
[492]	validation_0-merror:0.272827
[493]	validation_0-merror:0.272827
[494]	validation_0-merror:0.272827
[495]	validation_0-merror:0.27228
[496]	validation_0-merror:0.27228
[497]	validation_0-merror:0.272827
[498]	validation_0-merror:0.27228
[499]	validation_0-merror:0.273373


In [19]:
evaluate_model(X_train, y_train, X_test, y_test, xgb_model)

Training accuracy: 0.83, evaluation accuracy: 0.73


In [42]:
xgb_model = XGBClassifier(n_estimators=500).fit(X_train_all, y_train_all)

In [43]:
pickle.dump(xgb_model, open("ai_model_xgb.pkl","wb"))

random forest model

In [22]:
forest_model = RandomForestClassifier(random_state=1, n_estimators=100)
forest_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [23]:
evaluate_model(X_train, y_train, X_test, y_test, forest_model)

Training accuracy: 1.00, evaluation accuracy: 0.73


In [24]:
forest_model.fit(X_train_all, y_train_all)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [25]:
pickle.dump(forest_model, open("ai_model_forest.pkl","wb"))

In [26]:
# min_max_scaler = MinMaxScaler()
# np_scaled = min_max_scaler.fit_transform(df)
# df_normalized = pd.DataFrame(np_scaled)
# df_normalized.columns = feature_cols + ['Class']

In [27]:
# def make_crosses(X_train, X_validation, degree=2):
#   from sklearn.preprocessing import PolynomialFeatures
#   poly = PolynomialFeatures(degree=degree)
#   X_train_poly = poly.fit_transform(X_train)
#   X_validation_poly = poly.transform(X_validation)
#   return X_train_poly, X_validation_poly

In [50]:
mlp_model = MLPClassifier(solver='adam', alpha=1e-4, 
                    hidden_layer_sizes=(24, 24, 24, 24), 
                          random_state=1, verbose=True)

mlp_model.fit(X_train, y_train)

Iteration 1, loss = 1.79953014
Iteration 2, loss = 1.52667928
Iteration 3, loss = 1.32873253
Iteration 4, loss = 1.28672485
Iteration 5, loss = 1.25764412
Iteration 6, loss = 1.23454394
Iteration 7, loss = 1.21731290
Iteration 8, loss = 1.19579605
Iteration 9, loss = 1.16750399
Iteration 10, loss = 1.13219029
Iteration 11, loss = 1.08967387
Iteration 12, loss = 1.02936786
Iteration 13, loss = 0.95814345
Iteration 14, loss = 0.89041547
Iteration 15, loss = 0.84571754
Iteration 16, loss = 0.81547362
Iteration 17, loss = 0.79893303
Iteration 18, loss = 0.77699878
Iteration 19, loss = 0.76963686
Iteration 20, loss = 0.74904872
Iteration 21, loss = 0.73551862
Iteration 22, loss = 0.72120698
Iteration 23, loss = 0.70463922
Iteration 24, loss = 0.69446898
Iteration 25, loss = 0.68719453
Iteration 26, loss = 0.67387500
Iteration 27, loss = 0.65845593
Iteration 28, loss = 0.64368710
Iteration 29, loss = 0.63132856
Iteration 30, loss = 0.62329308
Iteration 31, loss = 0.61370057
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(24, 24, 24, 24), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=False)

In [51]:
evaluate_model(X_train, y_train, X_test, y_test, mlp_model)

Training accuracy: 0.99, evaluation accuracy: 0.63


In [52]:
mlp_model = MLPClassifier(solver='adam', alpha=1e-4, 
                    hidden_layer_sizes=(24, 24, 24, 24), 
                          random_state=1, verbose=True, max_iter=1000)

mlp_model.fit(X_train_all, y_train_all)

Iteration 1, loss = 1.76293850
Iteration 2, loss = 1.43177172
Iteration 3, loss = 1.29484560
Iteration 4, loss = 1.26298625
Iteration 5, loss = 1.23453085
Iteration 6, loss = 1.21094880
Iteration 7, loss = 1.18258405
Iteration 8, loss = 1.14310305
Iteration 9, loss = 1.08346755
Iteration 10, loss = 1.00843820
Iteration 11, loss = 0.93063833
Iteration 12, loss = 0.86687393
Iteration 13, loss = 0.83632784
Iteration 14, loss = 0.81500098
Iteration 15, loss = 0.80683912
Iteration 16, loss = 0.78507058
Iteration 17, loss = 0.77753593
Iteration 18, loss = 0.75997448
Iteration 19, loss = 0.74773567
Iteration 20, loss = 0.73694043
Iteration 21, loss = 0.72492454
Iteration 22, loss = 0.71477207
Iteration 23, loss = 0.70549178
Iteration 24, loss = 0.69165620
Iteration 25, loss = 0.68170070
Iteration 26, loss = 0.67375686
Iteration 27, loss = 0.65489900
Iteration 28, loss = 0.65143407
Iteration 29, loss = 0.64592755
Iteration 30, loss = 0.63224664
Iteration 31, loss = 0.62934795
Iteration 32, los

Iteration 260, loss = 0.11248694
Iteration 261, loss = 0.11815775
Iteration 262, loss = 0.11510973
Iteration 263, loss = 0.10958624
Iteration 264, loss = 0.10536429
Iteration 265, loss = 0.10290520
Iteration 266, loss = 0.10359602
Iteration 267, loss = 0.10228147
Iteration 268, loss = 0.10511731
Iteration 269, loss = 0.10254729
Iteration 270, loss = 0.09970292
Iteration 271, loss = 0.10263691
Iteration 272, loss = 0.10335739
Iteration 273, loss = 0.10234078
Iteration 274, loss = 0.09713669
Iteration 275, loss = 0.09611200
Iteration 276, loss = 0.09901696
Iteration 277, loss = 0.09563965
Iteration 278, loss = 0.09749752
Iteration 279, loss = 0.09849740
Iteration 280, loss = 0.09927140
Iteration 281, loss = 0.10768917
Iteration 282, loss = 0.09754622
Iteration 283, loss = 0.09943530
Iteration 284, loss = 0.11510918
Iteration 285, loss = 0.09603178
Iteration 286, loss = 0.08891212
Iteration 287, loss = 0.08716780
Iteration 288, loss = 0.08416010
Iteration 289, loss = 0.08746987
Iteration 

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(24, 24, 24, 24), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=False)

Save models to pkl

In [33]:
# model_p = pickle.load(open("ai_model.pkl","rb"))
game_state_4f = (np.array([8,280,630,-1,1,674,-3,7,667,-1,2,649,-3,2,662,0,3,662,-1,4,655,-1,5,608,2,7,1,8,288,630,-1,1,672,-3,7,668,-1,2,650,-3,2,661,0,3,657,-1,4,649,-1,5,602,2,7,1,8,296,629,-1,1,670,-3,7,669,-1,2,651,-3,2,660,0,3,651,-1,4,643,-1,5,596,2,7,1,8,296,628,-1,1,665,-3,7,667,-1,2,650,-3,2,658,0,3,647,-1,4,638,-1,5,589,2,7])
                 .reshape(1, -1))

game_state_1f = (np.array([25, -410, 308, 5, 1, -343, 483, 1, 2, -417, 592, 5, 2, -184, 589, 1, 2, -222, 275, 7, 2, -93, 143, 3, -2, -128, 549, 1, -3, -121, 399, 6, -3]).reshape(1, -1))

xgb_model.predict(game_state_1f)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32']
expected f56, f39, f63, f49, f41, f35, f84, f43, f79, f90,